In [2]:
import quadrantdetector.detector as qd
import numpy as np

from bokeh.palettes import *
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

from multiprocessing import Pool

output_notebook()

Loading BokehJS ...

In [3]:
n = 2000
detector_diameter = 10  # mm
detector_cell_gap = 0  # mm

In [4]:
detector = qd.create_detector(n, detector_diameter, detector_cell_gap)
print("Shape is", detector.shape, " and the max value is", detector.max())

Shape is (2000, 2000)  and the max value is 2.5000000000003914e-05


In [5]:
laser_sigma = 1
laser_x_coord = 0  # in Cartesian space
laser_y_coord = 0  # in Cartesian space

In [6]:
inscribed_laser = qd.laser(detector, laser_x_coord, laser_y_coord, laser_sigma)

In [7]:
print("Shape is", inscribed_laser.shape,
      "the max value is", inscribed_laser.max(),
      "the sum is", inscribed_laser.sum())

Shape is (2000, 2000) the max value is 3.978873416782049e-06 the sum is 0.9999962946331362


In [8]:
from scipy import integrate

def intensity(y, x, sigma):
    """ Computes the intensity of a Gaussian beam centered on the origin at the position (x,y)"""
    return (1 / (2 * np.pi * sigma**2)) * np.exp(-(x**2 + y**2)/(2 * sigma**2))


def total_signal(delta, sigma, R):
    """ Computes the theoretical intensity by sub-tracing off the signal lost due to the detector's
    finite size and the gap between the quadrants.
    """
    """
    signal = integrate.dblquad(intensity, 0, R, 0,
                               lambda x: np.sqrt(R**2 - x**2), args=(sigma,))[0]
    gap1 = integrate.dblquad(intensity, 0, delta/2,  0,
                             lambda x: np.sqrt(R**2 - x**2), args=(sigma,))[0]
    gap2 = integrate.dblquad(intensity, 0, delta/2, 0, delta/2,
                             args=(sigma,))[0]
    return 4 * signal - 8 * gap1 - 4 * gap2
    """
    
    return 4 * integrate.dblquad(intensity,
                             delta / 2, np.sqrt(R**2 - (delta / 2)**2),
                             delta / 2, lambda x: np.sqrt(R**2 - x**2), args=(sigma,))[0]

In [18]:
n = 2000

sigma_range = np.arange(0.1, 20, 0.1)

# Gap sizes are in mm.
gap_range = np.arange(1, 10, 0.5)


def plot_gap(new_gap):
    # Given this new gap, create a corresponding detector
    detector = qd.create_detector(n, detector_diameter, new_gap)
    
    # Then, for a range of Sigmas, calculate the sum signal using both our
    # integral and simulation.
    return ([qd.laser(detector, laser_x_coord, laser_y_coord, temp_sigma).sum()
             for temp_sigma in sigma_range],
            [total_signal(new_gap, temp_sigma, detector_diameter / 2)
             for temp_sigma in sigma_range])
    

with Pool(4) as t_pool:
    results = t_pool.map(plot_gap, gap_range)

p = figure(sizing_mode="scale_width", title="Simulated Detector Response",
           x_axis_label="Sigma Value", y_axis_label="Detector Signal")

for value_pair, new_gap in zip(results, gap_range):
    laser_signal, computed_laser_signal = value_pair
    p.line(sigma_range, laser_signal,
           alpha=0.5, muted_alpha=0.2, color="red", legend="simulated, " + str(new_gap))
    p.line(sigma_range, computed_laser_signal,
           alpha=0.5, muted_alpha=0.05, color="green", legend="computed, " + str(new_gap))

p.legend.location = "top_right"
p.legend.click_policy="mute"
show(p)